In [1]:
import torch
from torch import nn

from torchattacks.attacks.autoattack import AutoAttack
from torchattacks.attacks.bim import BIM
from torchattacks.attacks.fgsm import FGSM
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataloader import default_collate
from tqdm import tqdm

from torchvision import models

import os

from PizzaNet import PizzaNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

train_data_path = '../../data/train'
test_data_path = '../../data/test'

IMG_SIZE = 512

trans = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

train_data = ImageFolder(train_data_path, transform=trans)
train_dataset, valid_dataset = random_split(train_data, lengths=[0.8,0.2])

test_dataset = ImageFolder(test_data_path, transform=trans)
test_dataset_fancy = ImageFolder(test_data_path, transform=None)

In [3]:
model_path_load = f'saves{os.sep}latest_model_pizzanet.pickle'

model = PizzaNet(512)

model.load_state_dict(torch.load(model_path_load))
model.to(device)

PizzaNet(
  (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(128, 64, kernel_size=(5, 5), stride=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=230400, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
  (dropout): Dropout(p=0.35, inplace=False)
  (soft): Softmax(dim=1)
)

In [4]:
attack = FGSM(model)
#attack = AutoAttack(model, norm='Linf', n_classes=2, seed=10, verbose=True)

In [18]:
train_loader = DataLoader(train_dataset, batch_size=4)

adv_images = list()
adv_labels = list()

for i, (imgs, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):

    imgs = imgs.to(device)
    labels = labels.to(device)

    adv_images.append(attack(imgs, labels).to('cpu'))
    adv_labels.append(labels)

100%|██████████| 320/320 [00:33<00:00,  9.69it/s]


In [19]:
class attackDataset(Dataset):
    def __init__(self, images, labels):
        super(attackDataset, self).__init__()
        self.imgs = images
        self.labels = labels

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        return (self.imgs[index], self.labels[index])

adv_loader = DataLoader(attackDataset(torch.cat([*adv_images]), torch.cat([*adv_labels])), batch_size=4)

In [20]:
import torchmetrics

criterion = torch.nn.CrossEntropyLoss()

def evaluate(model, dataloader):
    
    f1 = torchmetrics.F1Score(task='binary', average='macro')
    model.eval()
    total_size = 0

    predicted = []
    targets = []
    
    losses = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            
            labels = labels.type(torch.LongTensor).to(device)
            predicted_labels = model(inputs.to(device)).squeeze()
            
            losses.append(float(criterion(predicted_labels, labels).to('cpu').item()))
            
            predicted.append(predicted_labels.to('cpu'))
            targets.append(labels.to('cpu'))
            
            total_size += labels.size(0)

    predicted = torch.cat([*predicted])
    targets = torch.cat([*targets])

    accuracy = (predicted.argmax(1) == targets).sum().item()/total_size

    f1score = f1(predicted.argmax(1), targets)
    return accuracy, f1score, sum(losses)/len(losses)

In [21]:
train_acc, train_f1, train_loss = evaluate(model, DataLoader(train_dataset, batch_size=4))
print('train accuracy: {} | f1-score: {} | loss: {}'.format(train_acc, train_f1, train_loss))

train accuracy: 0.86484375 | f1-score: 0.8571428656578064 | loss: 0.44866748647764326


In [22]:
adv_acc, adv_f1, adv_loss = evaluate(model, adv_loader)
print('ADV accuracy: {} | f1-score: {} | loss: {}'.format(adv_acc, adv_f1, adv_loss))

ADV accuracy: 0.609375 | f1-score: 0.4369369447231293 | loss: 0.6660731438547373
